In [1]:
# test_connection.py
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase

# .env 파일 로드
load_dotenv()

# 환경 변수 읽기
URI = os.getenv("NEO4J_URI")
USERNAME = os.getenv("NEO4J_USERNAME")
PASSWORD = os.getenv("NEO4J_PASSWORD")
DATABASE = os.getenv("NEO4J_DATABASE")

print(f"연결 시도 중...")
print(f"URI: {URI}")
print(f"Username: {USERNAME}")
print(f"Database: {DATABASE}")

# Neo4j 연결
try:
    driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))
    
    # 연결 테스트
    with driver.session(database=DATABASE) as session:
        result = session.run("RETURN 'Hello, Neo4j!' AS message")
        record = result.single()
        print(f"\n✅ 연결 성공!")
        print(f"메시지: {record['message']}")
    
    driver.close()
    
except Exception as e:
    print(f"\n❌ 연결 실패!")
    print(f"에러: {e}")

연결 시도 중...
URI: neo4j+s://0d1c14b7.databases.neo4j.io
Username: neo4j
Database: neo4j

✅ 연결 성공!
메시지: Hello, Neo4j!


In [ ]:
# setup_schema.py
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"),
    auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
)

def setup_schema():
    """제약조건과 인덱스 생성"""
    with driver.session(database=os.getenv("NEO4J_DATABASE")) as session:
        
        # 제약조건
        constraints = [
            "CREATE CONSTRAINT user_id IF NOT EXISTS FOR (u:User) REQUIRE u.userId IS UNIQUE",
            "CREATE CONSTRAINT product_id IF NOT EXISTS FOR (p:Product) REQUIRE p.productId IS UNIQUE",
            "CREATE CONSTRAINT category_name IF NOT EXISTS FOR (c:Category) REQUIRE c.name IS UNIQUE"
        ]
        
        # 인덱스
        indexes = [
            "CREATE INDEX user_name IF NOT EXISTS FOR (u:User) ON (u.name)",
            "CREATE INDEX user_email IF NOT EXISTS FOR (u:User) ON (u.email)",
            "CREATE INDEX product_name IF NOT EXISTS FOR (p:Product) ON (p.name)"
        ]
        
        print("📋 스키마 생성 중...")
        
        # 제약조건 생성
        for constraint in constraints:
            try:
                session.run(constraint)
                print(f"✅ {constraint[:50]}...")
            except Exception as e:
                print(f"⚠️  이미 존재: {constraint[:50]}...")
        
        # 인덱스 생성
        for index in indexes:
            try:
                session.run(index)
                print(f"✅ {index[:50]}...")
            except Exception as e:
                print(f"⚠️  이미 존재: {index[:50]}...")
        
        print("\n✅ 스키마 생성 완료!")
        
setup_schema()
driver.close()

In [ ]:
# testing cell
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"),
    auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
)
with driver.session(database=os.getenv("NEO4J_DATABASE")) as session:
    query = "MATCH (n) RETURN count(n) AS node_count"
    result = session.run(query, forceerror=["ex1", "ex2"])
    print(result.single()["node_count"])
driver.close()

In [ ]:
# generate_data.py
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker('ko_KR')  # 한국어 더미 데이터

driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"),
    auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
)

# 카테고리 목록
CATEGORIES = ['전자제품', '의류', '식품', '가구', '도서', '스포츠', '뷰티', '완구']

def clear_database():
    """기존 데이터 삭제"""
    with driver.session(database=os.getenv("NEO4J_DATABASE")) as session:
        print("🗑️  기존 데이터 삭제 중...")
        session.run("MATCH (n) DETACH DELETE n")
        print("✅ 삭제 완료!")

def create_categories():
    """카테고리 생성"""
    with driver.session(database=os.getenv("NEO4J_DATABASE")) as session:
        print(f"\n📂 카테고리 {len(CATEGORIES)}개 생성 중...")
        
        query = """
        UNWIND $categories AS category
        MERGE (c:Category {name: category})
        RETURN count(c) AS count
        """
        
        result = session.run(query, categories=CATEGORIES)
        count = result.single()['count']
        print(f"✅ {count}개 카테고리 생성 완료!")

def create_users(num_users=100):
    """사용자 생성"""
    with driver.session(database=os.getenv("NEO4J_DATABASE")) as session:
        print(f"\n👥 사용자 {num_users}명 생성 중...")
        
        users = []
        for i in range(1, num_users + 1):
            users.append({
                'userId': i,
                'name': fake.name(),
                'email': fake.email(),
                'phone': fake.phone_number(),
                'address': fake.address(),
                'joinedAt': fake.date_time_between(start_date='-2y', end_date='now').isoformat()
            })
        
        query = """
        UNWIND $users AS user
        CREATE (u:User {
            userId: user.userId,
            name: user.name,
            email: user.email,
            phone: user.phone,
            address: user.address,
            joinedAt: datetime(user.joinedAt)
        })
        """
        
        session.run(query, users=users)
        print(f"✅ {num_users}명 사용자 생성 완료!")

def create_products(num_products=200):
    """상품 생성"""
    with driver.session(database=os.getenv("NEO4J_DATABASE")) as session:
        print(f"\n🛍️  상품 {num_products}개 생성 중...")
        
        products = []
        for i in range(1, num_products + 1):
            products.append({
                'productId': i,
                'name': fake.catch_phrase(),
                'description': fake.text(max_nb_chars=100),
                'price': round(random.uniform(5000, 500000), -2),  # 5천원~50만원
                'stock': random.randint(0, 100),
                'category': random.choice(CATEGORIES)
            })
        
        query = """
        UNWIND $products AS product
        CREATE (p:Product {
            productId: product.productId,
            name: product.name,
            description: product.description,
            price: product.price,
            stock: product.stock
        })
        WITH p, product
        MATCH (c:Category {name: product.category})
        CREATE (p)-[:IN_CATEGORY]->(c)
        """
        
        session.run(query, products=products)
        print(f"✅ {num_products}개 상품 생성 완료!")

def create_purchases(num_purchases=500):
    """구매 관계 생성"""
    with driver.session(database=os.getenv("NEO4J_DATABASE")) as session:
        print(f"\n💰 구매 내역 {num_purchases}개 생성 중...")
        
        # 사용자와 상품 ID 범위 가져오기
        user_count = session.run("MATCH (u:User) RETURN count(u) AS count").single()['count']
        product_count = session.run("MATCH (p:Product) RETURN count(p) AS count").single()['count']
        
        purchases = []
        for _ in range(num_purchases):
            purchases.append({
                'userId': random.randint(1, user_count),
                'productId': random.randint(1, product_count),
                'quantity': random.randint(1, 5),
                'totalPrice': round(random.uniform(10000, 1000000), -2),
                'purchasedAt': fake.date_time_between(start_date='-1y', end_date='now').isoformat()
            })
        
        query = """
        UNWIND $purchases AS purchase
        MATCH (u:User {userId: purchase.userId})
        MATCH (p:Product {productId: purchase.productId})
        MERGE (u)-[r:PURCHASED]->(p)
        ON CREATE SET 
            r.quantity = purchase.quantity,
            r.totalPrice = purchase.totalPrice,
            r.purchasedAt = datetime(purchase.purchasedAt)
        """
        
        session.run(query, purchases=purchases)
        print(f"✅ {num_purchases}개 구매 내역 생성 완료!")

def create_reviews(num_reviews=300):
    """리뷰 생성"""
    with driver.session(database=os.getenv("NEO4J_DATABASE")) as session:
        print(f"\n⭐ 리뷰 {num_reviews}개 생성 중...")
        
        # 구매 관계 가져오기
        result = session.run("""
            MATCH (u:User)-[:PURCHASED]->(p:Product)
            RETURN u.userId AS userId, p.productId AS productId
            LIMIT $limit
        """, limit=num_reviews)
        
        reviews = []
        for record in result:
            reviews.append({
                'userId': record['userId'],
                'productId': record['productId'],
                'rating': random.randint(1, 5),
                'comment': fake.text(max_nb_chars=200),
                'reviewedAt': fake.date_time_between(start_date='-6m', end_date='now').isoformat()
            })
        
        query = """
        UNWIND $reviews AS review
        MATCH (u:User {userId: review.userId})
        MATCH (p:Product {productId: review.productId})
        MERGE (u)-[r:REVIEWED]->(p)
        SET r.rating = review.rating,
            r.comment = review.comment,
            r.reviewedAt = datetime(review.reviewedAt)
        """
        
        session.run(query, reviews=reviews)
        print(f"✅ {len(reviews)}개 리뷰 생성 완료!")

def print_statistics():
    """데이터 통계 출력"""
    with driver.session(database=os.getenv("NEO4J_DATABASE")) as session:
        print("\n" + "="*50)
        print("📊 데이터베이스 통계")
        print("="*50)
        
        # 노드 수
        stats = [
            ("User", "MATCH (u:User) RETURN count(u) AS count"),
            ("Product", "MATCH (p:Product) RETURN count(p) AS count"),
            ("Category", "MATCH (c:Category) RETURN count(c) AS count")
        ]
        
        for label, query in stats:
            count = session.run(query).single()['count']
            print(f"  {label}: {count:,}개")
        
        print()
        
        # 관계 수
        rels = [
            ("PURCHASED", "MATCH ()-[r:PURCHASED]->() RETURN count(r) AS count"),
            ("REVIEWED", "MATCH ()-[r:REVIEWED]->() RETURN count(r) AS count"),
            ("IN_CATEGORY", "MATCH ()-[r:IN_CATEGORY]->() RETURN count(r) AS count")
        ]
        
        for rel_type, query in rels:
            count = session.run(query).single()['count']
            print(f"  {rel_type}: {count:,}개")
        
        print("="*50 + "\n")

# 1. 기존 데이터 삭제
clear_database()

# 2. 데이터 생성
create_categories()
create_users(num_users=100)
create_products(num_products=200)
create_purchases(num_purchases=500)
create_reviews(num_reviews=300)

# 3. 통계 출력
print_statistics()

print("✅ 모든 작업 완료!")
driver.close()

In [ ]:
# query_examples.py
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"),
    auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
)

def run_query(query, parameters=None):
    """쿼리 실행 및 결과 출력"""
    with driver.session(database=os.getenv("NEO4J_DATABASE")) as session:
        result = session.run(query, parameters or {})
        return [record.data() for record in result]

# 예시 쿼리들
queries = {
    "1. 전체 사용자 수": """
        MATCH (u:User) 
        RETURN count(u) AS userCount
    """,
    
    "2. 상위 10개 인기 상품": """
        MATCH (p:Product)<-[r:PURCHASED]-()
        RETURN p.name AS product, 
               count(r) AS purchaseCount,
               avg(r.totalPrice) AS avgPrice
        ORDER BY purchaseCount DESC
        LIMIT 10
    """,
    
    "3. 카테고리별 상품 수": """
        MATCH (p:Product)-[:IN_CATEGORY]->(c:Category)
        RETURN c.name AS category, 
               count(p) AS productCount
        ORDER BY productCount DESC
    """,
    
    "4. 특정 사용자의 구매 이력": """
        MATCH (u:User {userId: 1})-[r:PURCHASED]->(p:Product)
        RETURN u.name AS user,
               p.name AS product,
               r.quantity AS quantity,
               r.totalPrice AS price,
               r.purchasedAt AS date
        ORDER BY date DESC
    """,
    
    "5. 평점 높은 상품 추천": """
        MATCH (p:Product)<-[r:REVIEWED]-()
        WITH p, avg(r.rating) AS avgRating, count(r) AS reviewCount
        WHERE reviewCount >= 3
        RETURN p.name AS product,
               avgRating,
               reviewCount
        ORDER BY avgRating DESC, reviewCount DESC
        LIMIT 10
    """
}

if __name__ == "__main__":
    print("\n" + "="*60)
    print("📊 샘플 쿼리 실행")
    print("="*60 + "\n")
    
    for title, query in queries.items():
        print(f"{title}")
        print("-" * 60)
        results = run_query(query)
        
        if results:
            # 결과 출력
            for i, record in enumerate(results, 1):
                print(f"  {i}. {record}")
        else:
            print("  결과 없음")
        
        print()
    
    driver.close()

# Understanding Neo4j - Graph Database Fundamentals

## Core Concepts

**Nodes** - Entities in your graph (e.g., User, Product, Category)
- Have labels (types): `(:User)`, `(:Product)`
- Have properties: `{name: "Alice", age: 30}`

**Relationships** - Connections between nodes
- Always have a direction: `(a)-[:KNOWS]->(b)`
- Have a type: `PURCHASED`, `REVIEWED`, `IN_CATEGORY`
- Can have properties: `[:PURCHASED {quantity: 2, date: "2024-01-01"}]`

**Properties** - Key-value pairs on nodes and relationships

## Cypher Query Language Patterns

```cypher
-- Find nodes
MATCH (n:Label) RETURN n

-- Find relationships  
MATCH (a)-[r:TYPE]->(b) RETURN a, r, b

-- Create nodes
CREATE (n:Label {property: value})

-- Create relationships
MATCH (a:Label1), (b:Label2)
CREATE (a)-[:RELATIONSHIP]->(b)
```

In [ ]:
# Setup: Create a reusable query runner
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv

load_dotenv()

driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"),
    auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
)

def run(query, params=None):
    """Execute a Cypher query and return results as list of dicts"""
    with driver.session(database=os.getenv("NEO4J_DATABASE")) as session:
        result = session.run(query, params or {})
        return [dict(record) for record in result]

print("✅ Driver ready")

## 1. Exploring the Graph Schema

First, let's see what types of nodes and relationships exist in our database.

In [ ]:
# Get all node labels in the database
labels = run("CALL db.labels()")
print("📦 Node Labels:")
for label in labels:
    print(f"  - {label['label']}")

# Get all relationship types
rel_types = run("CALL db.relationshipTypes()")
print("\n🔗 Relationship Types:")
for rel in rel_types:
    print(f"  - {rel['relationshipType']}")

# Get database schema visualization
schema = run("""
    CALL db.schema.visualization()
    YIELD nodes, relationships
    RETURN nodes, relationships
""")
print("\n📊 Schema loaded (visualize in Neo4j Browser for best view)")

## 2. Basic MATCH Patterns

The `MATCH` clause is the heart of Cypher - it describes patterns to find in your graph.

In [ ]:
# Pattern 1: Find all nodes of a type
users = run("MATCH (u:User) RETURN u.name AS name, u.email AS email LIMIT 5")
print("👤 First 5 Users:")
for u in users:
    print(f"  {u['name']} ({u['email']})")

# Pattern 2: Find nodes with specific properties
print("\n🔍 Find user by ID:")
user = run("MATCH (u:User {userId: 1}) RETURN u.name AS name, u.joinedAt AS joined")
print(f"  {user[0] if user else 'Not found'}")

# Pattern 3: Use WHERE for complex filtering
print("\n💰 Products over 100,000원:")
expensive = run("""
    MATCH (p:Product) 
    WHERE p.price > 100000 
    RETURN p.name AS name, p.price AS price 
    ORDER BY p.price DESC 
    LIMIT 5
""")
for p in expensive:
    print(f"  {p['name']}: ₩{p['price']:,.0f}")

## 3. Relationship Traversal

Relationships are what make graph databases powerful - they let you traverse connections naturally.

In [ ]:
# Traverse: User -> PURCHASED -> Product
print("🛒 What did User 1 purchase?")
purchases = run("""
    MATCH (u:User {userId: 1})-[r:PURCHASED]->(p:Product)
    RETURN p.name AS product, r.quantity AS qty, r.totalPrice AS total
    ORDER BY r.purchasedAt DESC
""")
for p in purchases[:5]:
    print(f"  {p['product']} (qty: {p['qty']}, ₩{p['total']:,.0f})")

# Traverse: Product -> IN_CATEGORY -> Category
print("\n📂 Products in '전자제품' category:")
products = run("""
    MATCH (p:Product)-[:IN_CATEGORY]->(c:Category {name: '전자제품'})
    RETURN p.name AS name, p.price AS price
    LIMIT 5
""")
for p in products:
    print(f"  {p['name']}: ₩{p['price']:,.0f}")

# Multi-hop traversal: User -> Product -> Category
print("\n🔄 Categories that User 1 shops in:")
categories = run("""
    MATCH (u:User {userId: 1})-[:PURCHASED]->(p:Product)-[:IN_CATEGORY]->(c:Category)
    RETURN DISTINCT c.name AS category, count(p) AS products_bought
    ORDER BY products_bought DESC
""")
for c in categories:
    print(f"  {c['category']}: {c['products_bought']} products")

## 4. Aggregations

Use `count()`, `sum()`, `avg()`, `collect()` to aggregate data across your graph.

In [ ]:
# Count purchases per user
print("📊 Top buyers:")
buyers = run("""
    MATCH (u:User)-[r:PURCHASED]->()
    RETURN u.name AS user, 
           count(r) AS purchases,
           sum(r.totalPrice) AS total_spent
    ORDER BY total_spent DESC
    LIMIT 5
""")
for b in buyers:
    print(f"  {b['user']}: {b['purchases']} purchases, ₩{b['total_spent']:,.0f}")

# Average rating per product
print("\n⭐ Best rated products:")
ratings = run("""
    MATCH (p:Product)<-[r:REVIEWED]-()
    WITH p, avg(r.rating) AS avg_rating, count(r) AS num_reviews
    WHERE num_reviews >= 2
    RETURN p.name AS product, 
           round(avg_rating * 10) / 10 AS rating,
           num_reviews AS reviews
    ORDER BY rating DESC
    LIMIT 5
""")
for r in ratings:
    print(f"  {r['product']}: {r['rating']}⭐ ({r['reviews']} reviews)")

# Collect into lists
print("\n📝 Products purchased by User 1 (as list):")
collected = run("""
    MATCH (u:User {userId: 1})-[:PURCHASED]->(p:Product)
    RETURN collect(p.name) AS products
""")
if collected:
    print(f"  {collected[0]['products'][:5]}...")

## 5. Recommendation Patterns

Graph databases excel at recommendations - finding connections through multiple hops.

In [ ]:
# Collaborative filtering: "Users who bought X also bought Y"
print("🎯 Collaborative Filtering - Similar purchases:")
print("   Users who bought what User 1 bought, also bought:")

similar = run("""
    // Find what User 1 purchased
    MATCH (u1:User {userId: 1})-[:PURCHASED]->(p:Product)
    
    // Find other users who purchased the same products
    MATCH (other:User)-[:PURCHASED]->(p)
    WHERE other <> u1
    
    // Find what those users also purchased (that User 1 hasn't)
    MATCH (other)-[:PURCHASED]->(rec:Product)
    WHERE NOT (u1)-[:PURCHASED]->(rec)
    
    RETURN rec.name AS recommended, 
           count(DISTINCT other) AS common_buyers
    ORDER BY common_buyers DESC
    LIMIT 5
""")
for s in similar:
    print(f"  {s['recommended']} ({s['common_buyers']} similar users)")

# Content-based: Same category recommendations
print("\n📦 Content-Based - Same category products:")
category_recs = run("""
    MATCH (u:User {userId: 1})-[:PURCHASED]->(p:Product)-[:IN_CATEGORY]->(c:Category)
    WITH u, c, count(p) AS affinity
    ORDER BY affinity DESC
    LIMIT 1
    
    MATCH (rec:Product)-[:IN_CATEGORY]->(c)
    WHERE NOT (u)-[:PURCHASED]->(rec)
    RETURN rec.name AS product, c.name AS category, rec.price AS price
    LIMIT 5
""")
for r in category_recs:
    print(f"  {r['product']} ({r['category']}) - ₩{r['price']:,.0f}")

## 6. Path Finding & Variable Length Patterns

Find paths of any length using `*` syntax - powerful for exploring connections.

In [ ]:
# Variable length path: Find connection between two users through purchases
print("🔗 Connection between User 1 and User 2 through shared products:")
paths = run("""
    MATCH path = (u1:User {userId: 1})-[:PURCHASED]->(p:Product)<-[:PURCHASED]-(u2:User {userId: 2})
    RETURN u1.name AS user1, p.name AS shared_product, u2.name AS user2
    LIMIT 3
""")
for p in paths:
    print(f"  {p['user1']} ←→ {p['shared_product']} ←→ {p['user2']}")

# Shortest path
print("\n📍 Shortest path between two users:")
shortest = run("""
    MATCH (u1:User {userId: 1}), (u2:User {userId: 10})
    MATCH path = shortestPath((u1)-[*]-(u2))
    RETURN [n IN nodes(path) | 
        CASE 
            WHEN n:User THEN 'User:' + n.name 
            WHEN n:Product THEN 'Product:' + n.name
            WHEN n:Category THEN 'Category:' + n.name
        END
    ] AS path_nodes
""")
if shortest:
    print(f"  {' → '.join(shortest[0]['path_nodes'])}")

# Count connections at each depth
print("\n📊 User 1's network at different depths:")
depths = run("""
    MATCH (u:User {userId: 1})-[:PURCHASED]->(p:Product)
    WITH count(DISTINCT p) AS direct_products
    
    MATCH (u:User {userId: 1})-[:PURCHASED]->(:Product)<-[:PURCHASED]-(other:User)
    WHERE other <> u
    WITH direct_products, count(DISTINCT other) AS connected_users
    
    RETURN direct_products, connected_users
""")
if depths:
    d = depths[0]
    print(f"  Depth 1: {d['direct_products']} products purchased")
    print(f"  Depth 2: {d['connected_users']} users with shared purchases")

## 7. Key Cypher Clauses Reference

| Clause | Purpose | Example |
|--------|---------|---------|
| `MATCH` | Find patterns | `MATCH (n:User)` |
| `WHERE` | Filter results | `WHERE n.age > 21` |
| `RETURN` | Output results | `RETURN n.name` |
| `CREATE` | Create nodes/relationships | `CREATE (n:User {name: 'Bob'})` |
| `MERGE` | Create if not exists | `MERGE (n:User {id: 1})` |
| `SET` | Update properties | `SET n.name = 'Alice'` |
| `DELETE` | Remove nodes/relationships | `DELETE n` |
| `WITH` | Chain query parts | `WITH n, count(*) AS c` |
| `ORDER BY` | Sort results | `ORDER BY n.name DESC` |
| `LIMIT` | Limit results | `LIMIT 10` |
| `UNWIND` | Expand lists to rows | `UNWIND [1,2,3] AS x` |

In [ ]:
# Cleanup: Close the driver connection
driver.close()
print("✅ Driver closed")